# MobileViT networks training for ensemble model


In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow import keras
from tensorflow.keras import layers
import time
import tensorflow as tf
import librosa, librosa.display


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Reshape, Conv2DTranspose, Dropout
from tensorflow.keras.layers import Reshape, MaxPooling2D, Softmax
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import SGD

from IPython import display
import random
from google.colab import drive


drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
%cd 'drive/MyDrive/DLOI_Project/'

/content/drive/.shortcut-targets-by-id/1K5OFKaoitJ_c6tsV6UYFW_Ts0E8dqdld/DLOI_Project


## Dataset

In [ ]:
# Download dataset
# !gdown 1-2_584NonvhhTgTEO0VBZrfooKxQujnT

In [ ]:
# from scipy.sparse import load_npz
# data = load_npz('/content/drive/MyDrive/DLOI_Project/audVisIdn.npz')

path_person_test='dataset/person_test.npy'
path_person_train='dataset/person_train.npy'
path_person_val='dataset/person_val.npy'

path_imgFrames_test='dataset/imgFrames_test.npy'
path_imgFrames_train='dataset/imgFrames_train.npy'
path_imgFrames_val='dataset/imgFrames_val.npy'

person_test = np.load(path_person_test, mmap_mode='r')
person_train = np.load(path_person_train, mmap_mode='r')
person_val = np.load(path_person_val, mmap_mode='r')

imgFrames_test = np.load(path_imgFrames_test, mmap_mode='r')
imgFrames_train = np.load(path_imgFrames_train, mmap_mode='r')
imgFrames_val = np.load(path_imgFrames_val, mmap_mode='r')

print('Labels:')
print(np.shape(person_test)) 
print(np.shape(person_train)) 
print(np.shape(person_val)) 

print('Faces images :')
print(np.shape(imgFrames_test)) 
print(np.shape(imgFrames_train)) 
print(np.shape(imgFrames_val)) 

nb_img_train, input_size, input_size, nb_channel = np.shape(imgFrames_train)

download_audio_images = False
 # Set to false to do the Mel spectogram again

if download_audio_images: # Load Mel spectogram directly
  S_DB_val_path='audio/AudioImages_Val.npy'
  S_DB_path='audio/AudioImages_Train.npy'
  S_DB_test_path='audio/AudioImages_Test.npy'

  S_DB_val = abs(np.load(S_DB_val_path, mmap_mode='r'))
  S_DB_test = abs(np.load(S_DB_test_path, mmap_mode='r'))
  S_DB = abs(np.load(S_DB_path, mmap_mode='r'))

  print('Audio images')
  print(np.shape(S_DB_val)) 
  print(np.shape(S_DB_test)) 
  print(np.shape(S_DB)) 

else: # Load Raw speeches

  path_audioTrs_test='dataset/audioTrs_test.npy'
  path_audioTrs_train='dataset/audioTrs_train.npy'
  path_audioTrs_val='dataset/audioTrs_val.npy'

  audioTrs_test = np.load(path_audioTrs_test, mmap_mode='r')
  audioTrs_train = np.load(path_audioTrs_train, mmap_mode='r')
  audioTrs_val = np.load(path_audioTrs_val, mmap_mode='r')  

  print('Audio speech')
  print(np.shape(audioTrs_test)) 
  print(np.shape(audioTrs_train)) 
  print(np.shape(audioTrs_val)) 


Labels:
(950,)
(2850,)
(950,)
Faces images :
(950, 224, 224, 3)
(2850, 224, 224, 3)
(950, 224, 224, 3)
Audio speech
(950, 56000)
(2850, 56000)
(950, 56000)


Rescale the labels between 0 and 49

In [ ]:
GT = list(np.unique(person_train))
new_id = [i for i in range(49)]
id_dict = {}
for k in range(len(GT)):
  id_dict[GT[k]] = new_id[k]

nb_class=49

person_train_GT = []
for i in range(person_train.shape[0]):
  person_train_GT.append(id_dict[person_train[i]]) 
person_train_GT = np.stack(person_train_GT)
output_train_class_onehot = tf.keras.utils.to_categorical(person_train_GT, nb_class)  # create one-hot encoded class

person_val_GT = []
for i in range(person_val.shape[0]):
  person_val_GT.append(id_dict[person_val[i]]) 
person_val_GT = np.stack(person_val_GT)
output_val_class_onehot = tf.keras.utils.to_categorical(person_val_GT, nb_class)  # create one-hot encoded class

person_test_GT = []
for i in range(person_test.shape[0]):
  person_test_GT.append(id_dict[person_test[i]]) 
person_test_GT = np.stack(person_test_GT)
output_test_class_onehot = tf.keras.utils.to_categorical(person_test_GT, nb_class)  # create one-hot encoded class
   
print('Labels GT:')
print(np.shape(person_test_GT)) 
print(np.shape(person_train_GT)) 
print(np.shape(person_val_GT)) 

Labels GT:
(950,)
(2850,)
(950,)


In [ ]:
if not download_audio_images:
  n_fft_opti = 2048
  hop_length_opti = 512 
  sr = 16000

  #training 
  S_DB = []
  for i in range(audioTrs_train.shape[0]):
    S = librosa.feature.melspectrogram(audioTrs_train[i], sr=sr, n_fft=n_fft_opti, hop_length=hop_length_opti)
    S_DB.append(np.stack([librosa.power_to_db(S, ref=np.max),librosa.power_to_db(S, ref=np.max),librosa.power_to_db(S, ref=np.max)], axis=2))

  S_DB = np.stack(S_DB, axis = 0)
    
  librosa.display.specshow(S_DB[2,:,:,0], sr=sr, x_axis='time', y_axis='mel')
  plt.colorbar(format='%+2.0f dB')

  # validation
  S_DB_val = []
  for i in range(audioTrs_val.shape[0]):
    S = librosa.feature.melspectrogram(audioTrs_val[i], sr=sr, n_fft=n_fft_opti, hop_length=hop_length_opti)
    S_DB_val.append(np.stack([librosa.power_to_db(S, ref=np.max),librosa.power_to_db(S, ref=np.max),librosa.power_to_db(S, ref=np.max)], axis=2))

  S_DB_val = np.stack(S_DB_val, axis = 0)

  plt.figure()
  librosa.display.specshow(S_DB_val[2,:,:,0], sr=sr, x_axis='time', y_axis='mel')
  plt.colorbar(format='%+2.0f dB')

  S_DB_test = []
  for i in range(audioTrs_test.shape[0]):
    S = librosa.feature.melspectrogram(audioTrs_test[i], sr=sr, n_fft=n_fft_opti, hop_length=hop_length_opti)
    S_DB_test.append(np.stack([librosa.power_to_db(S, ref=np.max),librosa.power_to_db(S, ref=np.max),librosa.power_to_db(S, ref=np.max)], axis=2))

  S_DB_test = np.stack(S_DB_test, axis = 0)

  # Scale audio mel spectrogram values to match cv image range (0 - 255)
  max_val = np.max([np.max(S_DB), np.max(S_DB_val), np.max(S_DB_test)])

  S_DB = 255/max_val*S_DB # range 0 to 255
  S_DB_val = 255/max_val*S_DB_val
  S_DB_test = 255/max_val*S_DB_test
else:
  fuse_id = []
  for i in range(S_DB.shape[0]):
    fuse_id.append(i%3 == 0)
  S_DB = S_DB[fuse_id]


In [ ]:
max_val = np.max([np.max(S_DB), np.max(S_DB_val), np.max(S_DB_test)])

S_DB = 255/max_val*S_DB # range 0 to 255
S_DB_val = 255/max_val*S_DB_val
S_DB_test = 255/max_val*S_DB_test
fuse_id = []
for i in range(S_DB.shape[0]):
  fuse_id.append(i%3 == 0)
S_DB = S_DB[fuse_id]

In [ ]:
def preprocess_spec(x, sr):
  S_DB = []
  for i in range(x.shape[0]):
    S = librosa.feature.melspectrogram(x[i], sr=sr, n_fft=2048, hop_length=512)
    S_DB.append(np.stack([librosa.power_to_db(S, ref=np.max),librosa.power_to_db(S, ref=np.max),librosa.power_to_db(S, ref=np.max)], axis=2))

  S_DB = np.stack(S_DB, axis = 0)
  return S_DB

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import time  
!wget https://upload.wikimedia.org/wikipedia/commons/f/f9/Zoorashia_elephant.jpg -O elephant.jpg
numTry = 1000
numParam = 25.6e6

with tf.device('/gpu:0'): # if using gpu for preprocessing /gpu:0
  model = ResNet50(weights='imagenet')  # num of params: 25.6 M
  img_path = 'elephant.jpg'
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  tic = time.perf_counter()
  for i in range(numTry):
    preds = model.predict(x)

  toc = time.perf_counter()
  totTime=toc-tic
  print("total time:",totTime)
  perRunTime = totTime/numTry
  paramSpeed=numParam/perRunTime
  print("number of parameters per second:",paramSpeed)
  tic = time.perf_counter()
  test_spec = preprocess_spec(audioTrs_train, 16000)
  toc = time.perf_counter()
  totTime=toc-tic
  perRunTime = totTime/(audioTrs_train.shape[0]) 
  print("Equivalent number of parameter for mel spectrogram: {:}".format(perRunTime*paramSpeed))
  # time your preprocessing operation for one sample and multiply with this
  # value to calculate parameter count equivalency 


--2022-05-31 08:36:28--  https://upload.wikimedia.org/wikipedia/commons/f/f9/Zoorashia_elephant.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168370 (164K) [image/jpeg]
Saving to: ‘elephant.jpg’

elephant.jpg        100%[===================>] 164.42K  --.-KB/s    in 0.04s   

2022-05-31 08:36:29 (4.56 MB/s) - ‘elephant.jpg’ saved [168370/168370]

total time: 66.85674916800002
number of parameters per second: 382908237.66604936
Equivalent number of parameter for mel spectrogram: 4754231.160716862


##Useful functions definition

In [ ]:
def lr_step_decay(epoch, lr):
    drop_rate = 0.1
    epochs_drop =  80.0
    return LEARNING_RATE * math.pow(drop_rate, math.floor(epoch/epochs_drop))

In [ ]:
def time_shift(aud, GT, shift_limit, nbr_augment):
  rolled = []
  rolled_GT = []
  for k in range(aud.shape[0]):
    rolled.append(aud[k])
    rolled_GT.append(GT[k])
    for i in range(nbr_augment):
      shift_rnd = random.random()
      if i == 0:
        shift_rnd = - shift_rnd
      rolled.append(np.roll(aud[k], shift=int(shift_rnd * shift_limit)))# * aud.shape[0])))
      rolled_GT.append(np.roll(GT[k], shift=int(shift_rnd * shift_limit)))# * GT.shape[0])))
    
  rolled = np.stack(rolled)
  rolled_GT = np.stack(rolled_GT)

  rolled = rolled#+np.random.normal(0,0.005,(rolled.shape[0],rolled.shape[1]))

  return rolled, rolled_GT

In [ ]:
def plot_loss_and_accuracy(history_sl):
  # model loss
  loss='categorical_crossentropy'
  metrics=['accuracy']

  plt.plot(history_sl.history['loss'])
  plt.plot(history_sl.history['val_loss'])
  plt.title('Model loss : ' + loss)
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Training', 'Validation'], loc='best')
  plt.show()
  plt.close()

  # model accuracy metric
  plt.plot(np.array(history_sl.history[metrics[0]]))
  plt.plot(np.array(history_sl.history['val_' + metrics[0]]))
  plt.title('Model accuracy metric : ' + metrics[0])
  plt.ylabel('Accuracy metric')
  plt.xlabel('Epoch')
  plt.legend(['Training', 'Validation'], loc='best')
  plt.show()
  plt.close()

## Define MobileViT network

In [ ]:
import tensorflow as tf
# ViT
!pip install -qq -U tensorflow-addons

import tensorflow_addons as tfa
import math

# Setting seed for reproducibiltiy
SEED = 42
keras.utils.set_random_seed(SEED)
from keras.applications import imagenet_utils
from tensorflow.keras import layers
from tensorflow import keras

import tensorflow_datasets as tfds
import tensorflow_addons as tfa

import math

tfds.disable_progress_bar()

# Values are from table 4.
patch_size = 4  # 2x2, for the Transformer blocks.
image_size = 256
expansion_factor = 2  # expansion factor for the MobileNetV2 blocks.

def conv_block(x, filters=16, kernel_size=3, strides=2):
    conv_layer = layers.Conv2D(
        filters, kernel_size, strides=strides, activation=tf.nn.swish, padding="same"
    )
    return conv_layer(x)



# Reference: https://git.io/JKgtC


def inverted_residual_block(x, expanded_channels, output_channels, strides=1):
    m = layers.Conv2D(expanded_channels, 1, padding="same", use_bias=False)(x)
    m = layers.BatchNormalization()(m)
    m = tf.nn.swish(m)

    if strides == 2:
        m = layers.ZeroPadding2D(padding=imagenet_utils.correct_pad(m, 3))(m)
    m = layers.DepthwiseConv2D(
        3, strides=strides, padding="same" if strides == 1 else "valid", use_bias=False
    )(m)
    m = layers.BatchNormalization()(m)
    m = tf.nn.swish(m)

    m = layers.Conv2D(output_channels, 1, padding="same", use_bias=False)(m)
    m = layers.BatchNormalization()(m)

    if tf.math.equal(x.shape[-1], output_channels) and strides == 1:
        return layers.Add()([m, x])
    return m



# Reference:
# https://keras.io/examples/vision/image_classification_with_vision_transformer/


def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.swish)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


def transformer_block(x, transformer_layers, projection_dim, num_heads=2):
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, x])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=[x.shape[-1] * 2, x.shape[-1]], dropout_rate=0.1,)
        # Skip connection 2.
        x = layers.Add()([x3, x2])

    return x


def mobilevit_block(x, num_blocks, projection_dim, strides=1):
    # Local projection with convolutions.
    local_features = conv_block(x, filters=projection_dim, strides=strides)
    local_features = conv_block(
        local_features, filters=projection_dim, kernel_size=1, strides=strides
    )

    # Unfold into patches and then pass through Transformers.
    num_patches = int((local_features.shape[1] * local_features.shape[2]) / patch_size)
    non_overlapping_patches = layers.Reshape((patch_size, num_patches, projection_dim))(
        local_features
    )
    global_features = transformer_block(
        non_overlapping_patches, num_blocks, projection_dim
    )

    # Fold into conv-like feature-maps.
    folded_feature_map = layers.Reshape((*local_features.shape[1:-1], projection_dim))(
        global_features
    )

    # Apply point-wise conv -> concatenate with the input features.
    folded_feature_map = conv_block(
        folded_feature_map, filters=x.shape[-1], kernel_size=1, strides=strides
    )
    local_global_features = layers.Concatenate(axis=-1)([x, folded_feature_map])

    # Fuse the local and global features using a convoluion layer.
    local_global_features = conv_block(
        local_global_features, filters=projection_dim, strides=strides
    )

    return local_global_features

def create_mobilevit(num_classes=49):
    inputs = keras.Input(shape=(256, 256, 3))
    x = layers.Rescaling(scale=1.0 / 255)(inputs)

    # Initial conv-stem -> MV2 block.
    x = conv_block(x, filters=16)
    x = inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=16
    )

    # Downsampling with MV2 block.
    x = inverted_residual_block(
        x, expanded_channels=16 * expansion_factor, output_channels=24, strides=2
    )
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=24
    )

    # First MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=24 * expansion_factor, output_channels=48, strides=2
    )
    x = mobilevit_block(x, num_blocks=2, projection_dim=64)

    # Second MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=64 * expansion_factor, output_channels=64, strides=2
    )
    x = mobilevit_block(x, num_blocks=4, projection_dim=80)

    # Third MV2 -> MobileViT block.
    x = inverted_residual_block(
        x, expanded_channels=80 * expansion_factor, output_channels=80, strides=2
    )
    x = mobilevit_block(x, num_blocks=3, projection_dim=96)
    x = conv_block(x, filters=512, kernel_size=1, strides=1)

    # Classification head.
    x = layers.GlobalAvgPool2D()(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs, outputs)



In [ ]:
import cv2
def upsample_image(dataset, chosen_input_size):
  # INTER_LINEAR or INTER_AREA
  return [cv2.resize(image, (chosen_input_size, chosen_input_size), interpolation=cv2.INTER_AREA) for image in dataset]

**Change input sizes to 256 x 256 x 3**

In [ ]:
x_train_img = np.stack(upsample_image(imgFrames_train, 256))
x_train_audio = np.stack(upsample_image(S_DB, 256))

In [ ]:
x_val_img = np.stack(upsample_image(imgFrames_val, 256))
x_val_audio = np.stack(upsample_image(S_DB_val, 256))

In [ ]:
x_test_img = np.stack(upsample_image(imgFrames_test, 256))
x_test_audio = np.stack(upsample_image(S_DB_test, 256))

## Faces and audio training from scratch

In [ ]:
mobilevit_img = create_mobilevit()

In [ ]:
mobilevit_audio = create_mobilevit()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import time  
!wget https://upload.wikimedia.org/wikipedia/commons/f/f9/Zoorashia_elephant.jpg -O elephant.jpg
numTry = 1000
numParam = 25.6e6

with tf.device('/gpu:0'): # if using gpu for preprocessing /gpu:0
  model = ResNet50(weights='imagenet')  # num of params: 25.6 M
  img_path = 'elephant.jpg'
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  tic = time.perf_counter()
  for i in range(numTry):
    preds = model.predict(x)

  toc = time.perf_counter()
  totTime=toc-tic
  perRunTime = totTime/numTry
  paramSpeed=numParam/perRunTime
  tic = time.perf_counter()
  _ = np.stack(upsample_image(imgFrames_train, 256))
  toc = time.perf_counter()
  totTime=toc-tic
  perRunTime = totTime/(audioTrs_train.shape[0]) 
  print("Equivalent number of parameter: {:}".format(perRunTime*paramSpeed))
  # time your preprocessing operation for one sample and multiply with this
  # value to calculate parameter count equivalency 


--2022-05-31 08:43:17--  https://upload.wikimedia.org/wikipedia/commons/f/f9/Zoorashia_elephant.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168370 (164K) [image/jpeg]
Saving to: ‘elephant.jpg’

elephant.jpg        100%[===================>] 164.42K  --.-KB/s    in 0.04s   

2022-05-31 08:43:17 (4.48 MB/s) - ‘elephant.jpg’ saved [168370/168370]

Equivalent number of parameter: 180945.2016642424


In [ ]:
augment_data = False

def data_augmentation_audio(data):
  b, width, height, c = np.shape(data)
  mask_length=20
  augmented_data=[]
  for i in np.arange(b):
    type_masking=random.randint(0, 1)

    if type_masking == 0:
      min_mask_width=random.randint(0, width-mask_length-1)
      temp=[np.zeros((height, 1)) if i<min_mask_width+mask_length and i>min_mask_width else x for i, x in enumerate(data[i])]
      augmented_data.append(temp)

    else:
      min_mask_height=random.randint(0, height-mask_length-1)
      temp=[np.zeros((width, 1)) if i<min_mask_height+mask_length and i>min_mask_height else x for i, x in enumerate(np.transpose(data[i], (1,0,2) ))]
      augmented_data.append(np.transpose(temp, (1,0,2)))
  return np.array(augmented_data)

if augment_data:
  S_DB[:,:,:,0]=data_augmentation_audio(np.expand_dims(S_DB[:,:,:,0], axis=3))[:,:,:,0]
  S_DB[:,:,:,1]=data_augmentation_audio(np.expand_dims(S_DB[:,:,:,1], axis=3))[:,:,:,0]
  S_DB[:,:,:,2]=data_augmentation_audio(np.expand_dims(S_DB[:,:,:,2], axis=3))[:,:,:,0]

In [ ]:
if augment_data:
  # Data augmentation
  data_augmentation = keras.Sequential(
      [
          layers.Normalization(),
          layers.RandomFlip("horizontal"),
          layers.RandomFlip("vertical"),
      ],
      name="data_augmentation",
  )
  # Compute the mean and the variance of the training data for normalization.
  if fuse:
    x_train_img = data_augmentation(x_train_img)

In [ ]:
label_smoothing_factor = 0.1
epochs = 40
BATCH_SIZE = 32
LEARNING_RATE = 0.002
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss_fn = keras.losses.CategoricalCrossentropy(label_smoothing=label_smoothing_factor)


def run_experiment_ensemble(model, x_train_ensemble,x_val_ensemble):
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )
    
    history = model.fit(
          x=x_train_ensemble,
          y=output_train_class_onehot,
          batch_size=BATCH_SIZE,
          epochs=epochs,
          validation_data=(x_val_ensemble, output_val_class_onehot),
          callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_step_decay, verbose=1)])

    return history

In [ ]:
mViT_audio = run_experiment_ensemble(mobilevit_audio, x_train_audio, x_val_audio)

In [ ]:
plot_loss_and_accuracy(mViT_audio)

In [ ]:
mViT_img = run_experiment_ensemble(mobilevit_img, x_train_img, x_val_img)

In [ ]:
plot_loss_and_accuracy(mViT_img)

In [ ]:
_, accuracy = mobilevit_img.evaluate(x_test_img, output_test_class_onehot)
print(f"Validation accuracy for images: {round(accuracy * 100, 2)}%")

In [ ]:
_, accuracy = mobilevit_audio.evaluate(x_test_audio, output_test_class_onehot)
print(f"Validation accuracy for images: {round(accuracy * 100, 2)}%")

## Load already trained network

In [ ]:
mobilevit_audio = keras.models.load_model("mobileAudio")
%cd audio/
mobilevit_img = keras.models.load_model("best")
%cd ..

/content/drive/.shortcut-targets-by-id/1K5OFKaoitJ_c6tsV6UYFW_Ts0E8dqdld/DLOI_Project/audio
/content/drive/.shortcut-targets-by-id/1K5OFKaoitJ_c6tsV6UYFW_Ts0E8dqdld/DLOI_Project


In [ ]:
_, accuracy = mobilevit_img.evaluate(x_test_img, output_test_class_onehot)
print(f"Validation accuracy for images: {round(accuracy * 100, 2)}%")

30/30 [==============================] - 5s 113ms/step - loss: 1.0891 - accuracy: 0.9147
Validation accuracy for images: 91.47%


In [ ]:
_, accuracy = mobilevit_audio.evaluate(x_test_audio, output_test_class_onehot)
print(f"Validation accuracy for images: {round(accuracy * 100, 2)}%")

30/30 [==============================] - 4s 121ms/step - loss: 1.4221 - accuracy: 0.8000
Validation accuracy for images: 80.0%


## Ensemble model performances

In [ ]:
# Training
proba_pred_train_img = mobilevit_img.predict(x_train_img)
proba_pred_train_audio = mobilevit_audio.predict(x_train_audio)
pred_train_img = np.argmax(proba_pred_train_img, axis=1) 
pred_train_audio = np.argmax(proba_pred_train_audio, axis=1) 

pred_train_ensemble = np.argmax(np.maximum(proba_pred_train_audio, proba_pred_train_img), axis=1) 
acc = np.sum(pred_train_ensemble==person_train_GT)/len(pred_train_ensemble)*100
print('Training : ensemble model accuracy {}'.format(acc))

Test : ensemble model accuracy 100.0


In [ ]:
# Validation
proba_pred_val_img = mobilevit_img.predict(x_val_img)
proba_pred_val_audio = mobilevit_audio.predict(x_val_audio)
pred_val_img = np.argmax(proba_pred_val_img, axis=1) 
pred_val_audio = np.argmax(proba_pred_val_audio, axis=1) 

pred_val_ensemble = np.argmax(np.maximum(proba_pred_val_audio, proba_pred_val_img), axis=1) 
acc = np.sum(pred_val_ensemble==person_val_GT)/len(pred_val_ensemble)*100
print('Validation : ensemble model accuracy {}'.format(acc))

Validation : ensemble model accuracy 93.57894736842105


In [ ]:
# Testing
proba_pred_test_img = mobilevit_img.predict(x_test_img)
proba_pred_test_audio = mobilevit_audio.predict(x_test_audio)
pred_test_img = np.argmax(proba_pred_test_img, axis=1) 
pred_test_audio = np.argmax(proba_pred_test_audio, axis=1) 

pred_test_ensemble = np.argmax(np.maximum(proba_pred_test_audio, proba_pred_test_img), axis=1) 
acc = np.sum(pred_test_ensemble==person_test_GT)/len(pred_test_ensemble)*100
print('Test : ensemble model accuracy {}'.format(acc))




Test : ensemble model accuracy 94.63157894736842
